## Building A Chatbot
In this file We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions



In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
groqapikey = os.getenv("groqapikey")


In [5]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groqapikey)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022147024AD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000221470256A0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.messages import HumanMessage, AIMessage
model.invoke([HumanMessage(content="hello my name is sunil mainali?")])


AIMessage(content="Hello Sunil Mainali! It's nice to meet you. 😊\n\nIs there anything I can help you with today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 18, 'total_tokens': 46, 'completion_time': 0.050909091, 'prompt_time': 0.00124679, 'queue_time': 0.25187538, 'total_time': 0.052155881}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f36581d0-aa16-438f-ae62-e98abcc3223d-0', usage_metadata={'input_tokens': 18, 'output_tokens': 28, 'total_tokens': 46})

In [11]:
model.invoke([HumanMessage(content="hello my name is sunil mainali?"),
              AIMessage(content="Hello Sunil Mainali! It's nice to meet you. 😊\n\nIs there anything I can help you with today?"),
              HumanMessage(content="hello whats my name?")])

AIMessage(content='You told me your name is Sunil Mainali!  \n\nIs there something else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 58, 'total_tokens': 83, 'completion_time': 0.045454545, 'prompt_time': 0.00226684, 'queue_time': 0.29508801, 'total_time': 0.047721385}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2deb2e76-1905-42cf-9a6d-37b69c7a941c-0', usage_metadata={'input_tokens': 58, 'output_tokens': 25, 'total_tokens': 83})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [18]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)


In [20]:
config={"configurable":{"session_id":"chat1"}}
config

{'configurable': {'session_id': 'chat1'}}

In [21]:
response=with_message_history.invoke([HumanMessage(content="hello my name is sunil mainali?")],config=config)
response

AIMessage(content="Hello Sunil Mainali, it's nice to meet you!\n\nIs there anything I can help you with today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 18, 'total_tokens': 45, 'completion_time': 0.049090909, 'prompt_time': 0.00125968, 'queue_time': 0.246428456, 'total_time': 0.050350589}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--4c3a3472-8ef1-4c1c-83e3-5b1b182fd941-0', usage_metadata={'input_tokens': 18, 'output_tokens': 27, 'total_tokens': 45})

In [22]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Sunil Mainali.  I remembered it from our first introduction! 😊 \n\n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 59, 'total_tokens': 91, 'completion_time': 0.058181818, 'prompt_time': 0.001789919, 'queue_time': 0.247936941, 'total_time': 0.059971737}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0b8b1de0-78ee-46a6-bdd9-f5ad03764690-0', usage_metadata={'input_tokens': 59, 'output_tokens': 32, 'total_tokens': 91})

In [23]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know!\n"

In [24]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is Vesh")],
    config=config1
)
response.content

"Hi Vesh, it's nice to meet you!  😊\n\nWhat can I do for you today?\n"

In [25]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Vesh!  \n\nI remember now.  😄 \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [27]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model
chain

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [28]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is sunil")]})

AIMessage(content="Hello Sunil, it's nice to meet you! \n\nI'm happy to help with any questions you have. Just ask away! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 32, 'total_tokens': 67, 'completion_time': 0.063636364, 'prompt_time': 0.00147835, 'queue_time': 0.24884272999999998, 'total_time': 0.065114714}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--558a1858-bd9a-48fc-9d43-6aa0f61e4da9-0', usage_metadata={'input_tokens': 32, 'output_tokens': 35, 'total_tokens': 67})

In [29]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [30]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is raj")],
    config=config
)

response

AIMessage(content="Hi Raj! It's nice to meet you. \n\nWhat can I do for you today? I'm happy to answer any questions you have to the best of my ability. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 31, 'total_tokens': 75, 'completion_time': 0.08, 'prompt_time': 0.00148216, 'queue_time': 0.24855521999999997, 'total_time': 0.08148216}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7c97cf80-bd41-4b41-a8c9-cebb2b34697e-0', usage_metadata={'input_tokens': 31, 'output_tokens': 44, 'total_tokens': 75})

In [31]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Raj!  I remember that you told me earlier.  \n\nIs there anything else I can help you with?  \n'

In [32]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [33]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Raj")],"language":"Nepali"})
response.content

'Namaste Raj! 😊  मलाई खुशी छ तपाईंले मलाई सम्बोधन गर्नुहुन्छ। के म तपाईंलाई कुनै किसिमको मद्दत गर्न सक्छु?  \n\n'

In [34]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [36]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Raj")],"language":"German"},
    config=config
)
repsonse.content

'Hallo Raj, schön, dich auch kennenzulernen! 👍\n\nWas kann ich für dich tun? \n'

In [37]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "German"},
    config=config,
)

In [38]:
response.content

'Dein Name ist Raj. 😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [42]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=90,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [43]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

'You like vanilla ice cream! 😊  🍦  \n\n\n\n'

In [45]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [46]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Your name is Bob! 😊  \n\n\n\n'

In [47]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked "what\'s 2 + 2"  😄  \n\n\n\n'